In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp
from rail.core.data import DATA_STORE
from rail.utils.catalog_utils import RomanRubinCatalogConfig
DS = DATA_STORE()

In [ ]:
from rail.raruma import utility_functions as raruma_util
from rail.raruma import plotting_functions as raruma_plot

Change this to be the root of the current PZ working area

In [ ]:
pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'

In [ ]:
d = tables_io.read(f"{pz_dir}/data/on_sky/dp1_v29.0.0_gold_all.hdf5")
pz = qp.read(f"{pz_dir}/projects/dp1/data/gold_baseline/dp1_all/output_estimate_knn.hdf5")

In [ ]:
#train = tables_io.sliceObj(d, slice(0, -1, 10))
train = d 
band_names = raruma_util.make_band_names('{band}_gaap1p0Mag', 'ugrizy')
mags = raruma_util.extract_data_to_2d_array(train,band_names)
mask = np.sum(np.isfinite(mags), axis=1) == 6
colors = raruma_util.adjacent_band_colors(mags)

In [ ]:
fluxes = raruma_util.mags_to_fluxes(mags, 31.4)

In [ ]:
pz_mode = pz.ancil['zmode']

In [ ]:
_ = raruma_plot.plot_colors_v_redshifts_with_templates(np.squeeze(pz_mode[mask]), colors[mask])

In [ ]:
_ = plt.hist(pz_mode[mask], bins=np.linspace(0., 3., 301))

In [ ]:
total_fluxes = fluxes[mask].sum(axis=1)

In [ ]:
fluxes

In [ ]:
n_obj = len(total_fluxes)

In [ ]:
picks = np.random.randint(n_obj, size=(n_obj))

In [ ]:
pick_fluxes = fluxes[picks]

In [ ]:
pick_totals = pick_fluxes.sum(axis=1)

In [ ]:
pick_weights = total_fluxes / pick_totals

In [ ]:
pick_weights

In [ ]:
contamination = (pick_fluxes.T*(pick_weights*0.01)).T

In [ ]:
new_fluxes = fluxes + contamination

In [ ]:
new_fluxes / fluxes

In [ ]:
new_mags = raruma_util.fluxes_to_mags(new_fluxes, 31.4)

In [ ]:
mags - new_mags

In [ ]:
new_colors = raruma_util.adjacent_band_colors(new_mags)
colors = raruma_util.adjacent_band_colors(mags)

In [ ]:
new_colors - colors